<a href="https://colab.research.google.com/github/Ashfagithub/twixia_internship_assignment_ashfa/blob/main/Twixia_Global_Internship_Assignment_Ashfa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Installing kaggle*

In [1]:
!pip install kaggle

In [2]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
!cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d anmolkumar/house-price-prediction-challenge

house-price-prediction-challenge.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
!unzip house-price-prediction-challenge.zip

Archive:  house-price-prediction-challenge.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [104]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, Ridge
from sklearn.preprocessing import PolynomialFeatures

In [61]:
!ls

brazil-clothing-marketplace-dataset.zip  sample_data
final_dataset_after_data_cleaning.csv	 sample_submission.csv
house-price-prediction-challenge.zip	 test.csv
kaggle.json				 train.csv
raw_dataset_from_scrapping.csv


In [62]:
df = pd.read_csv('./train.csv')

In [63]:
df_test = pd.read_csv('./test.csv')

In [64]:
df_test.shape

(68720, 11)

In [65]:
df.shape

(29451, 12)

In [66]:
df.head()

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS)
0,Owner,0,0,2,BHK,1300.236407,1,1,"Ksfc Layout,Bangalore",12.969910,77.597960,55.0
1,Dealer,0,0,2,BHK,1275.000000,1,1,"Vishweshwara Nagar,Mysore",12.274538,76.644605,51.0
2,Owner,0,0,2,BHK,933.159722,1,1,"Jigani,Bangalore",12.778033,77.632191,43.0
3,Owner,0,1,2,BHK,929.921143,1,1,"Sector-1 Vaishali,Ghaziabad",28.642300,77.344500,62.5
4,Dealer,1,0,2,BHK,999.009247,0,1,"New Town,Kolkata",22.592200,88.484911,60.5


In [67]:
df.columns.tolist()

['POSTED_BY',
 'UNDER_CONSTRUCTION',
 'RERA',
 'BHK_NO.',
 'BHK_OR_RK',
 'SQUARE_FT',
 'READY_TO_MOVE',
 'RESALE',
 'ADDRESS',
 'LONGITUDE',
 'LATITUDE',
 'TARGET(PRICE_IN_LACS)']

In [68]:
df_no_duplicates = df.drop_duplicates()
df=df_no_duplicates
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29050 entries, 0 to 29450
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   POSTED_BY              29050 non-null  object 
 1   UNDER_CONSTRUCTION     29050 non-null  int64  
 2   RERA                   29050 non-null  int64  
 3   BHK_NO.                29050 non-null  int64  
 4   BHK_OR_RK              29050 non-null  object 
 5   SQUARE_FT              29050 non-null  float64
 6   READY_TO_MOVE          29050 non-null  int64  
 7   RESALE                 29050 non-null  int64  
 8   ADDRESS                29050 non-null  object 
 9   LONGITUDE              29050 non-null  float64
 10  LATITUDE               29050 non-null  float64
 11  TARGET(PRICE_IN_LACS)  29050 non-null  float64
dtypes: float64(4), int64(5), object(3)
memory usage: 2.9+ MB


In [69]:
df.value_counts()

POSTED_BY  UNDER_CONSTRUCTION  RERA  BHK_NO.  BHK_OR_RK  SQUARE_FT    READY_TO_MOVE  RESALE  ADDRESS                    LONGITUDE  LATITUDE   TARGET(PRICE_IN_LACS)
Builder    0                   0     1        BHK        422.153370   1              0       Hoshangabad Road,Bhopal    22.750000  77.720000  10.9                     1
Owner      0                   0     1        BHK        646.678425   1              1       Amroli,Surat               21.239940  72.856520  11.0                     1
                                                         650.026001   1              1       Saravannapatti,Coimbatore  11.000000  76.966670  25.0                     1
                                                                                             Lokmanya Nagar,Palghar     22.691954  75.846950  25.0                     1
                                                                                             Katargam,Surat             21.232813  72.832454  25.0              

In [70]:
#df = df + df_test

In [71]:
df = df.drop(['ADDRESS','RERA'], axis=1)

In [72]:
df.head()

,POSTED_BY,UNDER_CONSTRUCTION,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS)
0,Owner,0,2,BHK,1300.236407,1,1,12.969910,77.597960,55.0
1,Dealer,0,2,BHK,1275.000000,1,1,12.274538,76.644605,51.0
2,Owner,0,2,BHK,933.159722,1,1,12.778033,77.632191,43.0
3,Owner,0,2,BHK,929.921143,1,1,28.642300,77.344500,62.5
4,Dealer,1,2,BHK,999.009247,0,1,22.592200,88.484911,60.5


In [73]:
# df['numbers'] = df.index

In [74]:
df.head()

,POSTED_BY,UNDER_CONSTRUCTION,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS)
0,Owner,0,2,BHK,1300.236407,1,1,12.969910,77.597960,55.0
1,Dealer,0,2,BHK,1275.000000,1,1,12.274538,76.644605,51.0
2,Owner,0,2,BHK,933.159722,1,1,12.778033,77.632191,43.0
3,Owner,0,2,BHK,929.921143,1,1,28.642300,77.344500,62.5
4,Dealer,1,2,BHK,999.009247,0,1,22.592200,88.484911,60.5


In [75]:
# label_and_numbers_column = df[['TARGET(PRICE_IN_LACS)','numbers']]
# label_and_numbers_column

In [76]:
# without_labels = df.drop('TARGET(PRICE_IN_LACS)',axis=1)
# without_labels

In [77]:
# for c in without_labels.columns:
#     if without_labels[c].dtype=='object':
#         lbl = LabelEncoder()
#         lbl.fit(list(without_labels[c].values))
#         without_labels[c] = lbl.transform(without_labels[c].values)

In [78]:
# without_labels.head()

In [79]:
# new_df = pd.merge(without_labels, label_and_numbers_column, on="numbers")

In [80]:
# new_df[30000:40000]

In [81]:
for c in df.columns:
    if df[c].dtype=='object':
        lbl = LabelEncoder()
        lbl.fit(list(df[c].values))
        df[c] = lbl.transform(df[c].values)

In [82]:
all_corr = df.corr().abs()['TARGET(PRICE_IN_LACS)'].sort_values(ascending = False)

In [83]:
all_corr

TARGET(PRICE_IN_LACS)    1.000000
SQUARE_FT                0.409571
RESALE                   0.203275
BHK_NO.                  0.113654
POSTED_BY                0.093575
UNDER_CONSTRUCTION       0.054618
READY_TO_MOVE            0.054618
LONGITUDE                0.030505
LATITUDE                 0.017813
BHK_OR_RK                0.004453
Name: TARGET(PRICE_IN_LACS), dtype: float64

In [84]:
X = df.drop('TARGET(PRICE_IN_LACS)',axis=1)

In [85]:
df.isnull().sum()

POSTED_BY                0
UNDER_CONSTRUCTION       0
BHK_NO.                  0
BHK_OR_RK                0
SQUARE_FT                0
READY_TO_MOVE            0
RESALE                   0
LONGITUDE                0
LATITUDE                 0
TARGET(PRICE_IN_LACS)    0
dtype: int64

In [86]:
y = df['TARGET(PRICE_IN_LACS)']

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=55)
X_train.shape, X_test.shape

((23240, 9), (5810, 9))

In [88]:
X_train

,POSTED_BY,UNDER_CONSTRUCTION,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,LONGITUDE,LATITUDE
6830,1,0,2,0,1225.114855,1,1,18.097689,74.735239
14252,1,0,1,0,650.042626,1,1,19.218498,72.992114
25443,2,0,2,0,1250.000000,1,1,12.968000,80.150200
17749,1,0,1,0,689.655172,1,1,18.966670,72.833330
15019,1,0,3,0,1910.176236,1,1,30.662283,76.822397
...,...,...,...,...,...,...,...,...,...
10258,2,0,2,0,950.118765,1,1,27.180000,78.020000
16386,1,1,2,0,486.017646,0,0,24.690280,78.418890
17507,1,1,1,0,760.836617,0,1,21.170000,72.830000
29366,1,1,2,0,992.025640,0,1,19.120000,72.910000


In [89]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [90]:
class Linear_Regression:

    def __init__(self, learning_rate=0.1, iterations=100):
        self.learning_rate = learning_rate
        self.iterations = iterations

    def fit(self, X, y):
        self.X = X
        self.y = y
        self.m, self.n = X.shape
        self.w = np.random.rand(self.n)
        self.b = 0

        for i in range(self.iterations):
            y_pred = np.dot(X, self.w) + self.b
            gW = - ( 2 * np.dot(X.T, (self.y - y_pred))) / self.m
            gb = - 2 * np.sum( self.y - y_pred ) / self.m
            self.w = self.w - self.learning_rate*gW
            self.b = self.b - self.learning_rate*gb

    def predict(self, X):
        return np.dot(X, self.w) + self.b

    def rmse(self, y, y_pred):
        return np.sqrt(np.mean(np.square(y - y_pred)))

    def r_squared(self, y, y_pred):
        return 1 - ((np.sum(np.square(y-y_pred)))/(np.sum(np.square(y-np.mean(y)))))

    def adjusted_r_squared(self, y, y_pred):
        return 1 - ((1-self.r_squared(y, y_pred))*(self.m - 1)/ (self.m - self.n - 1))

    @property
    def weights(self):
        return self.w

    @property
    def bias(self):
        return self.b

In [91]:
rf_model = RandomForestRegressor(random_state=42)
xgb_model = XGBRegressor(random_state=42)
custom_model = Linear_Regression()

In [92]:
rf_model.fit(X_test,y_test)

RandomForestRegressor(random_state=42)

In [93]:
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

X_train shape: (23240, 9)
y_train shape: (23240,)


In [105]:
class LinearRegressionCustom:

    def __init__(self, learning_rate=0.01, iterations=2000, verbose=False, early_stopping=False, validation_split=None,
                 regularization=None, alpha=0.001, poly_degree=None):
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.verbose = verbose
        self.early_stopping = early_stopping
        self.scaler = StandardScaler()
        self.w = None
        self.b = None
        self.regularization = regularization
        self.alpha = alpha
        self.poly_degree = poly_degree
        self.poly_features = None
        self.training_loss = []
        self.validation_loss = []
        self.best_weights = None
        self.best_loss = float('inf')
        self.patience = 50

        if validation_split:
            self.validation_split = validation_split
        else:
            self.validation_split = 0.2

    def fit(self, X, y):
        # Standardize the input data
        X = self.scaler.fit_transform(X)

        if self.poly_degree:
            self.poly_features = PolynomialFeatures(degree=self.poly_degree)
            X = self.poly_features.fit_transform(X)

        self.X = X
        self.y = y
        self.m, self.n = X.shape
        self.w = np.random.randn(self.n)
        self.b = 0

        if self.regularization == 'Lasso':
            model = Lasso(alpha=self.alpha)
        elif self.regularization == 'Ridge':
            model = Ridge(alpha=self.alpha)
        else:
            model = None

        if self.validation_split:
            split_idx = int((1 - self.validation_split) * self.m)
            X_train, y_train = X[:split_idx], y[:split_idx]
            X_val, y_val = X[split_idx:], y[split_idx:]
        else:
            X_train, y_train = X, y
            X_val, y_val = None, None

        for i in range(self.iterations):
            y_pred = np.dot(X_train, self.w) + self.b
            error = y_pred - y_train

            if model:
                reg_term = self.alpha * np.sign(self.w) if self.regularization == 'Lasso' else self.alpha * self.w
                gW = (-2 * np.dot(X_train.T, error) + reg_term) / self.m
            else:
                gW = -2 * np.dot(X_train.T, error) / self.m

            gb = -2 * np.sum(error) / self.m

            self.w -= self.learning_rate * gW
            self.b -= self.learning_rate * gb

            if self.verbose and i % 10 == 0:
                train_loss = self.rmse(y_train, y_pred)
                self.training_loss.append(train_loss)
                if X_val is not None and y_val is not None:
                    y_val_pred = self.predict(X_val)
                    val_loss = self.rmse(y_val, y_val_pred)
                    self.validation_loss.append(val_loss)
                    print(f"Iteration {i}: Train RMSE = {train_loss:.4f}, Validation RMSE = {val_loss:.4f}")

                    if val_loss < self.best_loss:
                        self.best_loss = val_loss
                        self.best_weights = self.w.copy()
                        patience_counter = 0
                    else:
                        patience_counter += 1
                        if patience_counter >= self.patience:
                            print(f"Early stopping at iteration {i}.")
                            self.w = self.best_weights
                            break
                else:
                    print(f"Iteration {i}: Train RMSE = {train_loss:.4f}")

    def predict(self, X):
        X = self.scaler.transform(X)

        if self.poly_features:
            X = self.poly_features.transform(X)

        return np.dot(X, self.w) + self.b

    def rmse(self, y, y_pred):
        return np.sqrt(np.mean(np.square(y - y_pred)))

    def r_squared(self, y, y_pred):
        return 1 - (np.sum(np.square(y - y_pred)) / np.sum(np.square(y - np.mean(y))))

    def adjusted_r_squared(self, y, y_pred):
        r_squared = self.r_squared(y, y_pred)
        return 1 - ((1 - r_squared) * (self.m - 1) / (self.m - self.n - 1))

    def evaluate(self, X_test, y_test):
        y_pred = self.predict(X_test)

        rmse = self.rmse(y_test, y_pred)
        r_squared = self.r_squared(y_test, y_pred)
        adjusted_r_squared = self.adjusted_r_squared(y_test, y_pred)

        return {
            'RMSE': rmse,
            'R-squared': r_squared,
            'Adjusted R-squared': adjusted_r_squared
        }

    @property
    def weights(self):
        return self.w

    @property
    def bias(self):
        return self.b


In [106]:
model = LinearRegressionCustom()
model.fit(X_train, y_train)
evaluation_results = model.evaluate(X_test, y_test)

print("Evaluation Results:")
print("RMSE:", evaluation_results['RMSE'])
print("R-squared:", evaluation_results['R-squared'])
print("Adjusted R-squared:", evaluation_results['Adjusted R-squared'])


Evaluation Results:
RMSE: 1.0711373423606917e+34
R-squared: -2.4400568711536695e+62
Adjusted R-squared: -2.4410022225028034e+62
